# GOTA API 예시

https://github.com/kniren/gota

## 필요 패키지

In [2]:
import (
    "fmt"
    "log"
    "os"
    "github.com/kniren/gota/dataframe"
    "github.com/kniren/gota/series"   
)

## 자료 생성

### 열별로(series) 타입을 지정하고 값을 넣어 생성한다.

In [7]:
df := dataframe.New(
	series.New([]string{"ㄱ", "ㄴ"}, series.String, "열1"),
	series.New([]int{1, 2}, series.Int, "열2"),
	series.New([]float64{3.0, 4.0}, series.Float, "열3"),
)

fmt.Println(df)

[2x3] DataFrame

    열1       열2    열3      
 0: ㄱ        1     3.000000
 1: ㄴ        2     4.000000
    <string> <int> <float> 

140
<nil>


### 배열을 사용해 행방향으로 자료 입력

In [10]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "나", "다", "라"},
        []string{"a", "4", "5.1", "true"},
        []string{"k", "5", "7.0", "true"},
        []string{"k", "4", "6.0", "true"},
        []string{"a", "2", "7.1", "false"},
    },
)
fmt.Println(df)

[4x4] DataFrame

    가        나     다        라     
 0: a        4     5.100000 true  
 1: k        5     7.000000 true  
 2: k        4     6.000000 true  
 3: a        2     7.100000 false 
    <string> <int> <float>  <bool>

236
<nil>


### 구조체에 담아서 행방향으로 자료 입력

In [16]:
type 사용자 struct {
	Name     string
	Age      int
	Accuracy float64
    ignored  bool // i소문자는 unexported 된다.
}
users := []User{
	{"아담", 17, 0.2, true},
	{"후안", 18, 0.8, true},
	{"아나", 22, 0.5, true},
}
df := dataframe.LoadStructs(users)
fmt.Println(df)

[3x4] DataFrame

    Name     Age   Accuracy LgoExport_ignored
 0: 아담       17    0.200000 true             
 1: 후안       18    0.800000 true             
 2: 아나       22    0.500000 true             
    <string> <int> <float>  <bool>           

260
<nil>


### 행(배열과 구조체)으로 타입지정하며 입력

불러오기 옵션 https://godoc.org/github.com/kniren/gota/dataframe#LoadOption

In [18]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "나", "다", "라"},
        []string{"a", "4", "5.1", "true"},
        []string{"k", "5", "7.0", "true"},
        []string{"k", "4", "6.0", "true"},
        []string{"a", "2", "7.1", "false"},
    },
    dataframe.DetectTypes(false),
    dataframe.DefaultType(series.Float),
    dataframe.WithTypes(map[string]series.Type{
        "가": series.String,
        "라": series.Bool,
    }),
)
fmt.Println(df)

[4x4] DataFrame

    가        나        다        라     
 0: a        4.000000 5.100000 true  
 1: k        5.000000 7.000000 true  
 2: k        4.000000 6.000000 true  
 3: a        2.000000 7.100000 false 
    <string> <float>  <float>  <bool>

254
<nil>


### 행 불러오기 옵션

https://godoc.org/github.com/kniren/gota/dataframe#LoadOption

func DefaultType(t series.Type) LoadOption
DefaultType sets the defaultType option for loadOptions.

func DetectTypes(b bool) LoadOption
DetectTypes sets the detectTypes option for loadOptions.

func HasHeader(b bool) LoadOption
HasHeader sets the hasHeader option for loadOptions.

func NaNValues(nanValues []string) LoadOption
NaNValues sets the nanValues option for loadOptions.

func Names(names ...string) LoadOption
Names sets the names option for loadOptions.

func WithDelimiter(b rune) LoadOption
WithDelimiter sets the csv delimiter other than ',', for example '\t'

func WithTypes(coltypes map[string]series.Type) LoadOption

### 맵으로 여러 묶음을 합치기

In [20]:
df := dataframe.LoadMaps(
    []map[string]interface{}{
        map[string]interface{}{
            "가": "a",
            "B": 1,
            "C": true,
            "D": 0,
        },
        map[string]interface{}{
            "A": "b",
            "B": 2,
            "C": true,
            "D": 0.5,
            "라": 0.2,
        },
    },
)

fmt.Println(df)

[2x6] DataFrame

    A        B     C      D        가        라       
 0:          1     true   0.000000 a        NaN     
 1: b        2     true   0.500000          0.200000
    <string> <int> <bool> <float>  <string> <float> 

234
<nil>


### 행열자료 불러오기 Matrix

In [40]:
import	"github.com/gonum/matrix/mat64"

data := []float64{1.2, -5.7, -2.4, 7.3}

// 2 by 2 matrix
a := mat64.NewDense(2, 2, data)

df := dataframe.LoadMatrix(a)

fmt.Println(df)

[2x2] DataFrame

    X0        X1       
 0: 1.200000  -5.700000
 1: -2.400000 7.300000 
    <float>   <float>  

114
<nil>


### io.Reader - cvs 형식 문자열 스트링버퍼

In [22]:
import "strings"

csvStr := `
Country,Date,Age,Amount,Id
"United States",2012-02-01,50,112.1,01234
"United States",2012-02-01,32,321.31,54320
"United Kingdom",2012-02-01,17,18.2,12345
"United States",2012-02-01,32,321.31,54320
"United Kingdom",2012-02-01,NA,18.2,12345
"United States",2012-02-01,32,321.31,54320
"United States",2012-02-01,32,321.31,54320
Spain,2012-02-01,66,555.42,00241
`
df := dataframe.ReadCSV(strings.NewReader(csvStr))

fmt.Println(df)

[8x5] DataFrame

    Country        Date       Age   Amount     Id   
 0: United States  2012-02-01 50    112.100000 1234 
 1: United States  2012-02-01 32    321.310000 54320
 2: United Kingdom 2012-02-01 17    18.200000  12345
 3: United States  2012-02-01 32    321.310000 54320
 4: United Kingdom 2012-02-01 NaN   18.200000  12345
 5: United States  2012-02-01 32    321.310000 54320
 6: United States  2012-02-01 32    321.310000 54320
 7: Spain          2012-02-01 66    555.420000 241  
    <string>       <string>   <int> <float>    <int>

548
<nil>


### io.Reader - json 형식 문자열 스트링버퍼

In [24]:
import "strings"
jsonStr := `[{"COL.2":1,"COL.3":3},{"COL.1":5,"COL.2":2,"COL.3":2},{"COL.1":6,"COL.2":3,"COL.3":1}]`
df := dataframe.ReadJSON(strings.NewReader(jsonStr))
fmt.Println(df)

[3x3] DataFrame

    COL.1 COL.2 COL.3
 0: NaN   1     3    
 1: 5     2     2    
 2: 6     3     1    
    <int> <int> <int>

128
<nil>


## 자료 선택 

### 행 - 서브셋 하위묶음 

In [4]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "나", "다", "라"},
        []string{"a", "4", "5.1", "true"},
        []string{"k", "5", "7.0", "true"},
        []string{"k", "4", "6.0", "true"},
        []string{"a", "2", "7.1", "false"},
    },
)

sub := df.Subset([]int{0,3})

fmt.Print(sub)

[2x4] DataFrame

    가        나     다        라     
 0: a        4     5.100000 true  
 1: a        2     7.100000 false 
    <string> <int> <float>  <bool>
165
<nil>


### 행 - 필터 서브셋보다 더 복잡한 묶음

fil 처럼 필터를 복함해서 쓰면 OR, 아래 fil2 처럼 기존 필터에 체인으로 적용하면 AND

In [19]:
fil := df.Filter(
    // 한 필터 내에서 조합되는 조건들은 OR
    dataframe.F{"가", series.Eq, "a"}, // OR
    dataframe.F{"나", series.Greater, 4},
) 
// 한 번 더 걸러냄. And 로
fil2 := fil.Filter( // And
    dataframe.F{"라", series.Eq, true},
)
fmt.Println(fil,fil2)

[2x4] DataFrame

    가        나     다        라     
 0: k        5     7.000000 true  
 1: a        2     7.100000 false 
    <string> <int> <float>  <bool>
 [1x4] DataFrame

    가        나     다        라     
 0: k        5     7.000000 true  
    <string> <int> <float>  <bool>

297
<nil>


### 열 - 셀렉트

선택 인덱스 종류 https://godoc.org/github.com/kniren/gota/dataframe#SelectIndexes

```
int              // Matches the given index number
[]int            // Matches all given index numbers
[]bool           // Matches all columns marked as true
string           // Matches the column with the matching column name
[]string         // Matches all columns with the matching column names
Series [Int]     // Same as []int
Series [Bool]    // Same as []bool
Series [String]  // Same as []string
```

In [23]:
sel1 := df.Select([]int{1, 2})
sel2 := df.Select([]string{"가", "라"})
fmt.Println(sel1,sel2)

[4x2] DataFrame

    나     다       
 0: 4     6.000000
 1: 5     7.000000
 2: 3     7.000000
 3: 2     7.100000
    <int> <float> 
 [4x2] DataFrame

    가        라     
 0: z        false 
 1: k        true  
 2: x        false 
 3: a        false 
    <string> <bool>

278
<nil>


## 자료 정렬

###  열 - 어레인지

In [29]:
sorted가 := df.Arrange(
    dataframe.Sort("가"),    // Sort in ascending order
)
sorted라 := df.Arrange(
    dataframe.RevSort("라"), // Sort in descending order
)
fmt.Println(sorted가,sorted라)

[4x4] DataFrame

    가        나     다        라     
 0: a        2     7.100000 false 
 1: k        5     7.000000 true  
 2: x        3     7.000000 false 
 3: z        4     6.000000 false 
    <string> <int> <float>  <bool>
 [4x4] DataFrame

    가        나     다        라     
 0: k        5     7.000000 true  
 1: z        4     6.000000 false 
 2: x        3     7.000000 false 
 3: a        2     7.100000 false 
    <string> <int> <float>  <bool>

472
<nil>


## 값 업데이트 및 추가

### 행 - Set 으로 같은 형태의 값을 덮어씌운다

단점이 있다면 원본을 바로 수정해 버린다.

In [15]:
df.Set(
    // 번째와 2번째 행을
    []int{0, 2},
    // 다음 데이터 프레임값으로 업데이트
    dataframe.LoadRecords(
        [][]string{
            []string{"가", "나", "다", "라"},
            []string{"z", "4", "6.0", "false"},
            []string{"x", "3", "7.0", "false"},
        },
    ),
)
fmt.Print(df)

[4x4] DataFrame

    가        나     다        라     
 0: z        4     6.000000 false 
 1: k        5     7.000000 true  
 2: x        3     7.000000 false 
 3: a        2     7.100000 false 
    <string> <int> <float>  <bool>
235
<nil>


그러니 복사해서 쓰자.

In [ ]:
df2 := df.Copy

df2.Set(
    // 번째와 2번째 행을
    []int{0, 2},
    // 다음 데이터 프레임값으로 업데이트
    dataframe.LoadRecords(
        [][]string{
            []string{"가", "나", "다", "라"},
            []string{"z", "4", "6.0", "false"},
            []string{"x", "3", "7.0", "false"},
        },
    ),
)
fmt.Print(df)

### 열 - Mutate 수정 및 추가

이것은 원본을 수정하지 않는다. 굿

In [34]:
// 다 열의 시리즈 타입과 값을 변경
mut := df.Mutate(
    series.New([]string{"a", "b", "c", "d"}, series.String, "다"),
)
// 새로운 마 열 추가
mut2 := df.Mutate(
    series.New([]string{"a", "b", "c", "d"}, series.String, "마"),
)

fmt.Println(mut,mut2)

[4x4] DataFrame

    가        나     다        라     
 0: z        4     a        false 
 1: k        5     b        true  
 2: x        3     c        false 
 3: a        2     d        false 
    <string> <int> <string> <bool>
 [4x5] DataFrame

    가        나     다        라      마       
 0: z        4     6.000000 false  a       
 1: k        5     7.000000 true   b       
 2: x        3     7.000000 false  c       
 3: a        2     7.100000 false  d       
    <string> <int> <float>  <bool> <string>

528
<nil>


### 열 이름 하나 바꾸기

In [97]:
a := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"좌", "9", "7.1", "false"},
    },
)

a = a.Rename("나", "좌키")

fmt.Println(a)

[3x4] DataFrame

    가        나     다        라     
 0: ㄱ        4     5.100000 true  
 1: ㄴ        5     7.000000 true  
 2: 좌        9     7.100000 false 
    <string> <int> <float>  <bool>

207
<nil>


### 열 이름 모두 바꾸기

이것도 원본 자료 자체를 바꿔버린다. 그리 좋은 메소드는 아닌 듯..

In [108]:
a := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"좌", "9", "7.1", "false"},
    },
)

// cannot use a.SetNames("강", "낭", "당", "랑") (value of type error) as github.com/kniren/gota/dataframe.DataFrame value in assignment
// a = a.SetNames("강", "낭", "당", "랑")
// a is dataframe and is not equal to nil

a.SetNames("강", "낭", "당", "랑")

import "reflect"

reflect.TypeOf(a)


fmt.Println(a)

[3x4] DataFrame

    강        낭     당        랑     
 0: ㄱ        4     5.100000 true  
 1: ㄴ        5     7.000000 true  
 2: 좌        9     7.100000 false 
    <string> <int> <float>  <bool>

207
<nil>


##  조인

### 이너조인

![](https://community.toadworld.com/cfs-filesystemfile/__key/communityserver-components-secureimagefileviewer/communityserver-wikis-components-files-00-00-00-00-06/JP1C_5F00_151.gif_2D00_550x0.png
)

리차드와 우드가 매뉴팩처링 값을 가지고 있는 것을 유의. 값이 없으면 복사해버린다.

In [53]:
좌 := dataframe.LoadRecords(
    [][]string{
        []string{"가", "키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"ㄷ", "4", "6.0", "true"},
        []string{"ㄹ", "2", "7.1", "false"},
        []string{"좌측에만있는요소", "9", "7.1", "false"},
    },
)
우 := dataframe.LoadRecords(
    [][]string{
        []string{"키", "사", "라"},
        []string{"10", "우측에만 있는 요소", "true"},
        []string{"4", "2", "false"},
        []string{"2", "8", "false"},
        []string{"5", "9", "false"},
    },
)
join := 우.InnerJoin(좌, "키")
fmt.Println(join)

[4x6] DataFrame

    키     사        라_0    가        다        라_1   
 0: 4     2        false  ㄱ        5.100000 true  
 1: 4     2        false  ㄷ        6.000000 true  
 2: 2     8        false  ㄹ        7.100000 false 
 3: 5     9        false  ㄴ        7.000000 true  
    <int> <string> <bool> <string> <float>  <bool>

344
<nil>


df2 는 라가 하나 뿐이라서 . 가_1 과 사 가 df의 세개의 트루줄마다 복제된다.

### Leftjoin

좌측에 있는 것중 우축에 있는 것만 가져온다.

![](https://www.w3resource.com/w3r_images/sqlite-left-join-pictorial-example1.gif)

In [51]:
좌 := dataframe.LoadRecords(
    [][]string{
        []string{"가", "키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"ㄷ", "4", "6.0", "true"},
        []string{"ㄹ", "2", "7.1", "false"},
        []string{"좌측에만있는요소", "9", "7.1", "false"},
    },
)
우 := dataframe.LoadRecords(
    [][]string{
        []string{"키", "사", "라"},
        []string{"10", "우측에만 있는 요소", "true"},
        []string{"4", "2", "false"},
        []string{"2", "8", "false"},
        []string{"5", "9", "false"},
    },
)
// 좌측 키값에는 1이 없다. 즉 결과는 우측 키값인
join := 좌.LeftJoin(우, "키")
fmt.Println(join)

[5x6] DataFrame

    키     가        다        라_0    사        라_1   
 0: 4     ㄱ        5.100000 true   2        false 
 1: 5     ㄴ        7.000000 true   9        false 
 2: 4     ㄷ        6.000000 true   2        false 
 3: 2     ㄹ        7.100000 false  8        false 
 4: 9     좌측에만있는요소 7.100000 false  NaN      NaN   
    <int> <string> <float>  <bool> <string> <bool>

411
<nil>


### Rightjoin

우측 키에 있는 것만 좌측과 결합해서 가져온다.

In [54]:
좌 := dataframe.LoadRecords(
    [][]string{
        []string{"가", "키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"ㄷ", "4", "6.0", "true"},
        []string{"ㄹ", "2", "7.1", "false"},
        []string{"좌측에만있는요소", "9", "7.1", "false"},
    },
)
우 := dataframe.LoadRecords(
    [][]string{
        []string{"키", "사", "라"},
        []string{"1", "우측에만 있는 요소", "true"},
        []string{"4", "2", "false"},
        []string{"2", "8", "false"},
        []string{"5", "9", "false"},
    },
)
// 좌측 키값에는 1이 없다. 즉 결과는 우측 키값인
join := 좌.RightJoin(우, "키")
fmt.Println(join)

DataFrame error: join keys not specified
41
<nil>


### outerjoin

![](https://www.w3schools.com/sql/img_fulljoin.gif)

The FULL OUTER JOIN keyword return all records when there is a match in either left (table1) or right (table2) table records.
두 테이블의 어느 한쪽이라도 맞는 모든 레코드를 반환한다.

In [69]:
좌 := dataframe.LoadRecords(
    [][]string{
        []string{"가", "키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "4", "6.0", "true"},
        []string{"ㄷ", "5", "7.0", "true"},
        []string{"ㄹ", "2", "7.1", "false"},
        []string{"좌측에만있는요소", "9", "7.1", "false"},
    },
)
우 := dataframe.LoadRecords(
    [][]string{
        []string{"키", "사", "라"},
        []string{"1", "우측에만 있는 요소", "true"},
        []string{"4", "2", "false"},
        []string{"2", "8", "false"},
        []string{"5", "9", "false"},
    },
)
// 좌측 키값에는 1이 없다. 즉 결과는 우측 키값인
join := 좌.OuterJoin(우, "키")
fmt.Println(join)

[6x6] DataFrame

    키     가        다        라_0    사          라_1   
 0: 4     ㄱ        5.100000 true   2          false 
 1: 4     ㄴ        6.000000 true   2          false 
 2: 5     ㄷ        7.000000 true   9          false 
 3: 2     ㄹ        7.100000 false  8          false 
 4: 9     좌측에만있는요소 7.100000 false  NaN        NaN   
 5: 1     NaN      NaN      NaN    우측에만 있는 요소 true  
    <int> <string> <float>  <bool> <string>   <bool>

494
<nil>


### Crossjoin

연관이 없는 두 테이블의 모든 열을 조합하는 것.
수학의 경우의수, 조합할 때 그 조합이다.

![](https://www.w3resource.com/w3r_images/sqlite-cross-join-pictorial-presentation.gif)

In [67]:
좌 := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"좌", "9", "7.1", "false"},
    },
)
우 := dataframe.LoadRecords(
    [][]string{
        []string{"우키", "사", "라"},
        []string{"1", "우", "true"},
        []string{"4", "2", "false"},
        []string{"2", "8", "false"},
    },
)
// 모든 경우의 수인 조합
join := 좌.CrossJoin(우)
fmt.Println(join)

joinSort := join.Arrange(dataframe.Sort("우키"))
fmt.Println(joinSort)

[9x7] DataFrame

    가        좌키    다        라_0    우키    사        라_1   
 0: ㄱ        4     5.100000 true   1     우        true  
 1: ㄱ        4     5.100000 true   4     2        false 
 2: ㄱ        4     5.100000 true   2     8        false 
 3: ㄴ        5     7.000000 true   1     우        true  
 4: ㄴ        5     7.000000 true   4     2        false 
 5: ㄴ        5     7.000000 true   2     8        false 
 6: 좌        9     7.100000 false  1     우        true  
 7: 좌        9     7.100000 false  4     2        false 
 8: 좌        9     7.100000 false  2     8        false 
    <string> <int> <float>  <bool> <int> <string> <bool>

[9x7] DataFrame

    가        좌키    다        라_0    우키    사        라_1   
 0: ㄱ        4     5.100000 true   1     우        true  
 1: ㄴ        5     7.000000 true   1     우        true  
 2: 좌        9     7.100000 false  1     우        true  
 3: ㄱ        4     5.100000 true   2     8        false 
 4: ㄴ        5     7.000000 true   2     8        fal

## 처리하기

###  합치기

In [43]:
좌 := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"좌", "9", "7.1", "false"},
    },
)
우 := dataframe.LoadRecords(
    [][]string{
        []string{"우키", "사", "라"},
        []string{"1", "우", "true"},
        []string{"4", "2", "false"},
        []string{"2", "8", "false"},
    },
)

cbind := 좌.CBind(우)
fmt.Println(cbind)

[3x7] DataFrame

    가        좌키    다        라_0    우키    사        라_1   
 0: ㄱ        4     5.100000 true   1     우        true  
 1: ㄴ        5     7.000000 true   4     2        false 
 2: 좌        9     7.100000 false  2     8        false 
    <string> <int> <float>  <bool> <int> <string> <bool>

DataFrame error: rbind: column names are not compatible
56
<nil>


In [45]:
좌 := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"좌", "9", "7.1", "false"},
    },
)
우 := dataframe.LoadRecords(
    [][]string{
        []string{"좌키", "가", "다", "라"},
        []string{"1", "우","5.1", "true"},
        []string{"4", "2", "7.0", "false"},
        []string{"2", "8", "7.1", "false"},
    },
)

rbind := 좌.RBind(우)
fmt.Println(rbind)

[6x4] DataFrame

    가        좌키    다        라     
 0: ㄱ        4     5.100000 true  
 1: ㄴ        5     7.000000 true  
 2: 좌        9     7.100000 false 
 3: 우        1     5.100000 true  
 4: 2        4     7.000000 false 
 5: 8        2     7.100000 false 
    <string> <int> <float>  <bool>

316
<nil>


### 분리하기

특정열을 분리시켜서 시리즈 타입으로 반환받는다.

In [53]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"1", "4", "5.1", "6"},
        []string{"2", "5", "7.0", "4"},
        []string{"6", "9", "7.1", "10"},
    },
)

a := df.Col("좌키")

fmt.Println(a)

import "reflect"

reflect.TypeOf(a)

[4 5 9]
series.Series


시리즈 구조체

In [ ]:
type Series struct {
    Name string // The name of the series

    Err error // If there are errors they are stored here
    // contains filtered or unexported fields
}

### 제거하기

특정열을 제거한 결과를 돌려준다.

In [60]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"1", "4", "5.1", "6"},
        []string{"2", "5", "7.0", "4"},
        []string{"6", "9", "7.1", "10"},
    },
)

a := df.Drop("좌키")

fmt.Println(a)

[3x3] DataFrame

    가     다        라    
 0: 1     5.100000 6    
 1: 2     7.000000 4    
 2: 6     7.100000 10   
    <int> <float>  <int>

149
<nil>


### 함수 적용

In [36]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"1", "4", "5.1", "6"},
        []string{"2", "5", "7.0", "4"},
        []string{"6", "9", "7.1", "10"},
    },
)
mean := func(s series.Series) series.Series {
    floats := s.Float()
    sum := 0.0
    for _, f := range floats {
        sum += f
    }
    return series.Floats(sum / float64(len(floats)))
}
fmt.Println(df.Capply(mean))


fmt.Println(df.Rapply(mean))

[1x4] DataFrame

    가        좌키       다        라       
 0: 3.000000 6.000000 6.400000 6.666667
    <float>  <float>  <float>  <float> 

[3x1] DataFrame

    X0      
 0: 4.025000
 1: 4.500000
 2: 8.025000
    <float> 

83
<nil>


## 변환 내보내기

### 데이터프레임을 문자열로 [][]string 변환

In [126]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"1", "4", "5.1", "6"},
        []string{"2", "5", "7.0", "4"},
        []string{"6", "9", "7.1", "10"},
    },
)

str := df.String()

fmt.Println(str)

// 문자열이니 조작이 가능함.
fmt.Println(str[:5])


[3x4] DataFrame

    가     좌키    다        라    
 0: 1     4     5.100000 6    
 1: 2     5     7.000000 4    
 2: 6     9     7.100000 10   
    <int> <int> <float>  <int>

[3x4]
6
<nil>


### 데이터프레임을 배열 혹은 슬라이스로 [][]string 변환

In [71]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"1", "4", "5.1", "6"},
        []string{"2", "5", "7.0", "4"},
        []string{"6", "9", "7.1", "10"},
    },
)

fmt.Println(df.Records())


[[가 좌키 다 라] [1 4 5.100000 6] [2 5 7.000000 4] [6 9 7.100000 10]]
75
<nil>


### 데이터프레임을 CSV 로 변환

인자로 io.writer 를 받는데... 아래처럼 파일 접속을 넘겼는데도 되네.

https://gobyexample.com/writing-files

In [122]:
import (
    "io/ioutil"
    "os"
    "reflect"
)

df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"1", "4", "5.1", "6"},
        []string{"2", "5", "7.0", "4"},
        []string{"6", "9", "7.1", "10"},
    },
)

f, err := os.Create("./df.csv")
defer f.Close()

// 해더 false 하면 자료만 쓴다.
// df.WriteCSV(f, dataframe.WriteHeader(false))
df.WriteCSV(f)

fmt.Println(reflect.TypeOf(f))


*os.File
9
<nil>


### 데이터프레임을 JSON 으로 변환

In [115]:
import (
    "io/ioutil"
    "os"
    "reflect"
)

df := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"1", "4", "5.1", "6"},
        []string{"2", "5", "7.0", "4"},
        []string{"6", "9", "7.1", "10"},
    },
)

f, err := os.Create("./df.json")
defer f.Close()


df.WriteJSON(f)

fmt.Println(reflect.TypeOf(f))

*os.File
9
<nil>


## 내부 정보 확인

### 행열 개수 확인

In [67]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"A", "B", "C", "D"},
        []string{"a", "4", "5.1", "true"},
        []string{"b", "4", "6.0", "true"},
        []string{"c", "3", "6.0", "false"},
        []string{"a", "2", "7.1", "false"},
    },
)

ncol := df.Ncol()
nrow := df.Nrow()
fmt.Println(ncol,nrow)

4 4
4
<nil>


### 열 이름 확인

In [65]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"A", "B", "C", "D"},
        []string{"a", "4", "5.1", "true"},
        []string{"b", "4", "6.0", "true"},
        []string{"c", "3", "6.0", "false"},
        []string{"a", "2", "7.1", "false"},
    },
)

// row 4(index 3), column  3(index 2)
names := df.Names()
fmt.Println(names)

[A B C D]
10
<nil>


### 열 타입 확인

In [109]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"A", "B", "C", "D"},
        []string{"a", "4", "5.1", "true"},
        []string{"b", "4", "6.0", "true"},
        []string{"c", "3", "6.0", "false"},
        []string{"a", "2", "7.1", "false"},
    },
)

// row 4(index 3), column  3(index 2)
types := df.Types()
fmt.Println(types)

[string int float bool]
24
<nil>


### 특정 원소 확인

In [62]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"A", "B", "C", "D"},
        []string{"a", "4", "5.1", "true"},
        []string{"b", "4", "6.0", "true"},
        []string{"c", "3", "6.0", "false"},
        []string{"a", "2", "7.1", "false"},
    },
)

// row 4(index 3), column  3(index 2)
el := df.Elem(3,2)
fmt.Println(el)

7.100000
9
<nil>


### 통계 요약 출력

In [55]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"A", "B", "C", "D"},
        []string{"a", "4", "5.1", "true"},
        []string{"b", "4", "6.0", "true"},
        []string{"c", "3", "6.0", "false"},
        []string{"a", "2", "7.1", "false"},
    },
)
fmt.Println(df.Describe())

[7x5] DataFrame

    column   A        B        C        D       
 0: mean     -        3.250000 6.050000 0.500000
 1: stddev   -        0.957427 0.818535 0.577350
 2: min      a        2.000000 5.100000 0.000000
 3: 25%      -        2.000000 5.100000 0.000000
 4: 50%      -        3.000000 6.000000 0.000000
 5: 75%      -        4.000000 6.000000 1.000000
 6: max      c        4.000000 7.100000 1.000000
    <string> <string> <float>  <float>  <float> 

459
<nil>


### 몇차원(dimensions)인지 출력

In [58]:
df := dataframe.LoadRecords(
    [][]string{
        []string{"A", "B", "C", "D"},
        []string{"a", "4", "5.1", "true"},
        []string{"b", "4", "6.0", "true"},
        []string{"c", "3", "6.0", "false"},
        []string{"a", "2", "7.1", "false"},
    },
)
fmt.Println(df.Dims())

4 4
4
<nil>


## 수행 체이닝

In [28]:
a := dataframe.LoadRecords(
    [][]string{
        []string{"가", "좌키", "다", "라"},
        []string{"ㄱ", "4", "5.1", "true"},
        []string{"ㄴ", "5", "7.0", "true"},
        []string{"좌", "9", "7.1", "false"},
    },
)

a = a.Rename("나", "좌키").
    Filter(dataframe.F{"나", "<", 9}).
    Filter(dataframe.F{"라", "==", "true"}).
    Select([]string{"가","나", "다"}).
    Subset([]int{0})

fmt.Println(a)

[1x3] DataFrame

    가        나     다       
 0: ㄱ        4     5.100000
    <string> <int> <float> 

110
<nil>
